In [1]:
# The ultimate target feature: time from one stop to another

# Imports
import pandas as pd
import numpy as np
from datetime import date, datetime
from patsy import dmatrices
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.tree import export_graphviz, DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.tree import export_graphviz 
from sklearn import tree
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 
from statsmodels.formula.api import ols

# Read csv file into a dataframe.
df = pd.read_csv('clean142.csv')

# 1.0 Model Analysis

## 1.1 Random Forest Regression

In [2]:
#journey pattern ID here just represents direction - we can switch for direction for user input
#took out RDS event as not right to simply include one event - not consist for all events at RDS in Jan
#stop order is essentially the number of times that bus stopped - we will not know that
#speed - uses the stop order to work out time travelling (includes time journey started) / stop order (position) - we will not know that
#therefore is the best we can do in real time is to record the journey times/distances as speeds for each scheduled run?
y, X = dmatrices('Time_To_Travel ~ Day_Of_Week + Time_Bin_Start +  Scheduled_Speed_Per_Stop + Wind_Speed +  Temperature  + Holiday + Stops_To_Travel + Stop_Sequence', df, return_type="dataframe") 
y = np.ravel(y)
X

,Intercept,Day_Of_Week,Time_Bin_Start,Scheduled_Speed_Per_Stop,Wind_Speed,Temperature,Holiday,Stops_To_Travel,Stop_Sequence
0,1.0,2.0,16110.0,2.02381,14.0,5.5,1.0,0.0,42.0
1,1.0,2.0,17000.0,2.02381,14.0,5.5,1.0,0.0,42.0
2,1.0,2.0,17100.0,2.02381,14.0,5.5,1.0,0.0,42.0
3,1.0,2.0,17111.0,2.02381,14.0,5.5,1.0,0.0,42.0
4,1.0,2.0,16110.0,2.02381,14.0,5.5,1.0,0.0,42.0
5,1.0,2.0,17000.0,2.02381,14.0,5.5,1.0,0.0,42.0
6,1.0,2.0,17100.0,2.02381,14.0,5.5,1.0,0.0,42.0
7,1.0,2.0,17111.0,2.02381,14.0,5.5,1.0,0.0,42.0
8,1.0,1.0,8111.0,2.02381,14.0,5.5,0.0,0.0,1.0
9,1.0,1.0,16110.0,2.02381,14.0,5.5,0.0,0.0,42.0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=33) 

In [4]:
scaler = preprocessing.StandardScaler().fit(X_train)

In [5]:
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=10))

In [6]:
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}

In [7]:
clf = GridSearchCV(pipeline, hyperparameters, cv=8)

In [8]:
clf.fit(X_train, y_train)

GridSearchCV(cv=8, error_score='raise',
       estimator=Pipeline(steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'randomforestregressor__max_features': ['auto', 'sqrt', 'log2'], 'randomforestregressor__max_depth': [None, 5, 3, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [9]:
pred = clf.predict(X_test)

In [10]:
r2_score(y_test, pred)

0.96222909658715594

Good, closer to 1 the better

In [11]:
mean_squared_error(y_test, pred)

79.506399767114061

Good, closer to 0 the better.

This saves the model for later.

In [12]:
joblib.dump(clf, '../flask_app/static/rf_regressor.pkl')

['../flask_app/static/rf_regressor.pkl']

In [13]:
X_test.head()

,Intercept,Day_Of_Week,Time_Bin_Start,Scheduled_Speed_Per_Stop,Wind_Speed,Temperature,Holiday,Stops_To_Travel,Stop_Sequence
144,1.0,3.0,8110.0,2.02381,14.0,9.9,0.0,0.0,1.0
29,1.0,0.0,17110.0,2.02381,14.0,2.7,0.0,0.0,42.0
81,1.0,4.0,8110.0,2.02381,8.0,5.5,0.0,0.0,1.0
184,1.0,4.0,17110.0,2.02381,4.0,4.7,0.0,0.0,42.0
8,1.0,1.0,8111.0,2.02381,14.0,5.5,0.0,0.0,1.0


This can then be reloaded to be used in the flask app. As seen below.

In [14]:
clf2 = joblib.load('../flask_app/static/rf_regressor.pkl')

In [15]:
predictions = list(clf2.predict(X_test))
predictions

modified=[]

for i in predictions:
    modified.append(i)
# print(modified)

X_test["Prediction"]=0
X_test["Prediction"]=modified
X_test.head()

/Users/katerooney/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/katerooney/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,Intercept,Day_Of_Week,Time_Bin_Start,Scheduled_Speed_Per_Stop,Wind_Speed,Temperature,Holiday,Stops_To_Travel,Stop_Sequence,Prediction
144,1.0,3.0,8110.0,2.02381,14.0,9.9,0.0,0.0,1.0,0.0
29,1.0,0.0,17110.0,2.02381,14.0,2.7,0.0,0.0,42.0,98.7
81,1.0,4.0,8110.0,2.02381,8.0,5.5,0.0,0.0,1.0,0.0
184,1.0,4.0,17110.0,2.02381,4.0,4.7,0.0,0.0,42.0,94.1
8,1.0,1.0,8111.0,2.02381,14.0,5.5,0.0,0.0,1.0,0.0


In [16]:
rfc = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1)

In [17]:
df.shape

(185, 44)

## To Do

Date is in the model as a variable making significant impact as cannot group random forest by dates - to confirm why it makes an impact (improves score about 7% but we should  understand is this equiv to groupby or coincidence or a proxy for events).

Getting a memory error with feature importance so for now using the linear results to guide - issue with rain: clearly important - we know that - and humidity is important, but no rain columns are making any impact. See commented out code below.

In [18]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=True, random_state=1,
            verbose=0, warm_start=False)

In [19]:
pd.DataFrame({'feature': X_train.columns, 'importance':rfc.feature_importances_})

,feature,importance
0,Intercept,0.000000
1,Day_Of_Week,0.135769
2,Time_Bin_Start,0.354779
3,Scheduled_Speed_Per_Stop,0.000000
4,Wind_Speed,0.165134
5,Temperature,0.215200
6,Holiday,0.018756
7,Stops_To_Travel,0.000000
8,Stop_Sequence,0.110362


# Decision tree for single tree - not going to work here but gives visual of the tree structure

In [20]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4,min_samples_leaf=5)
clf = clf.fit(X_train,y_train)

In [21]:
tree.export_graphviz(clf, out_file='tree.dot', feature_names = X_train.columns)
!dot -Tpng tree.dot > tree.png
from IPython.display import Image 
Image(filename='tree.png')

/bin/sh: dot: command not found


In [22]:
from sklearn import metrics
def measure_performance(X,y,clf, show_accuracy=True,show_classification_report=True, show_confussion_matrix=True):
    
    y_pred=clf.predict(X)
    
    if show_accuracy:
        print ("accuracy",metrics.accuracy_score(y, y_pred))
        
    if show_classification_report:
        print ("classification",metrics.classification_report(y,y_pred))
            
    if show_confussion_matrix:
        print ("confusion matrix",metrics.confusion_matrix(y,y_pred))
        
measure_performance(X_train,y_train,clf)

accuracy 0.486486486486
classification              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00        49
       74.0       0.00      0.00      0.00         1
       75.0       0.25      1.00      0.40         3
       80.0       0.00      0.00      0.00         1
       81.0       0.00      0.00      0.00         1
       82.0       0.00      0.00      0.00         1
       83.0       0.00      0.00      0.00         1
       84.0       0.16      1.00      0.27         3
       85.0       0.00      0.00      0.00         2
       86.0       0.00      0.00      0.00         1
       87.0       0.00      0.00      0.00         4
       88.0       0.00      0.00      0.00         2
       89.0       0.17      0.44      0.25         9
       90.0       0.00      0.00      0.00         6
       91.0       0.20      0.50      0.29         6
       92.0       0.30      0.75      0.43         4
       93.0       0.00      0.00      0.00         3
      

/Users/katerooney/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# K nearest neighbours

In [23]:
import random
import math
from numpy.random import permutation

# Randomly shuffle the index of df.
random_indices = permutation(df.index)
# Set a cutoff for how many items we want in the test set (in this case 1/3 of the items)
test_cutoff = math.floor(len(df)/3)
# Generate the test set by taking the first 1/3 of the randomly shuffled indices.
test = df.loc[random_indices[1:test_cutoff]]
# Generate the train set with the rest of the data.
train = df.loc[random_indices[test_cutoff:]]

In [24]:
x_columns = ['speed', 'stop_order','DayOfWeek', 'HourOfDay', 'MinsOfHour', 'Direction_north']
y_column = ['time_bins']

from sklearn.neighbors import KNeighborsRegressor
# Create the knn model.
# Look at the five closest neighbors.
knn = KNeighborsRegressor(n_neighbors=5)
# Fit the model on the training data.
knn.fit(train[x_columns], train[y_column])
# Make point predictions on the test set using the fit model.
predictions = knn.predict(test[x_columns])


KeyError: "['speed' 'stop_order' 'DayOfWeek' 'HourOfDay' 'MinsOfHour'\n 'Direction_north'] not in index"

In [ ]:
# Get the actual values for the test set.
actual = test[y_column]

# Compute the mean squared error of our predictions.
mse = (((predictions - actual) ** 2).sum()) / len(predictions)


In [ ]:
mse

# Linear model

In [ ]:
import statsmodels.formula.api as sm

# Train all continuous features.
check1 = sm.ols(formula='Time_To_Travel ~ Day_Of_Week + Time_Bin_Start + Scheduled_Speed_Per_Stop +  Temperature + Wind_Speed +  Stops_To_Travel + Stop_Sequence', data=df).fit()
# Print the weights learned for each feature.
print(check1.summary())

# Logistic Regression Model with Bins

In [ ]:
# Training the model with original selection of four features:

logreg = sm.logit(formula='Time_To_Travel ~ Day_Of_Week + Time_Bin_Start + Scheduled_Speed_Per_Stop +  Temperature + Wind_Speed +  Stops_To_Travel + Stop_Sequence', data=df).fit()

# Print the weights learned for each feature.
print(logreg.params)